# I/O

## 1. Input from the code

In [1]:
a = input('Please, enter something: ')
print('You entered:', a)

Please, enter something: a
You entered: a


## 2. Input from the OS prompt

In [3]:
!cat argparse_example.py

import argparse

parser = argparse.ArgumentParser(description='This program bla, bla, bla ...')
parser.add_argument('-i', '--input', help='Input file name', default='stdin')

args = parser.parse_known_args()[0]

if args.input:
    print("Using input file", args.input)



In [4]:
!python argparse_example.py -h

usage: argparse_example.py [-h] [-i INPUT]

This program bla, bla, bla ...

optional arguments:
  -h, --help            show this help message and exit
  -i INPUT, --input INPUT
                        Input file name


In [6]:
!python argparse_example.py -i abc

Using input file abc


## Another posibility is to use Fire:

In [7]:
%%writefile using_fire.py
import fire

class A():
    def do_something(self, x, y):
        print('Doing something with:', y, x)
        
fire.Fire(A)

Writing using_fire.py


In [9]:
!python using_fire.py

Type:        A
String form: <__main__.A object at 0x7f159afdf668>
File:        ~/YAPT/using_fire.py

Usage:       using_fire.py 
             using_fire.py do-something


In [10]:
!python using_fire.py do-something hola caracola

Doing something with: caracola hola


## 3. Disk

In [11]:
import io

Write some integers:

In [12]:
# The legacy way:
file = open('/tmp/some_integers_1.txt', 'w')
file.write('{}\n'.format(1))
file.write('{}\n'.format(2))
file.write('{}\n'.format(3))
file.close()

In [13]:
!cat /tmp/some_integers_1.txt

1
2
3


In [14]:
# The modern (pythonic) alternative:
with io.open('/tmp/some_integers_2.txt', 'w') as file:
    file.write('{}\n'.format(1))
    file.write('{}\n'.format(2))
    file.write('{}\n'.format(3))

In [15]:
file.closed

True

In [17]:
!cat /tmp/some_integers_2.txt

1
2
3


Reading the file:

In [18]:
# The classic alternative:
file = io.open('/tmp/some_integers_1.txt', 'r')
while True:
    line = file.readline()
    if not line:
        break
#for i in range(3):
    print(int(line))
file.close()

1
2
3


In [19]:
# The pythonic alternarive:
with io.open('/tmp/some_integers_2.txt', 'r') as file:
    for line in file:
        print(int(line))

1
2
3


In [20]:
file.closed

True

### Binary VS Text modes

In the previous examples we have used the text mode for handling files, which basicaly means that we have using strings which can be edited with a text editor. On the other hand, binary files can not edited with a text editor, but are shorter than text files when storing numbers. The following code is the alternative to the previous one using binary mode.

In [21]:
import struct # See https://docs.python.org/3/library/struct.html#format-characters

In [22]:
with io.open('/tmp/some_integers_2.bin', 'wb') as file:
    file.write(struct.pack('h', 1)) # 2 bytes, signed int
    file.write(struct.pack('h', 2)) # 2 bytes, signed int
    file.write(struct.pack('i', 3)) # 4 bytes, signed int

In [23]:
with io.open('/tmp/some_integers_2.bin', 'rb') as file:
    print(struct.unpack('h', file.read(struct.calcsize('h')))[0])
    print(struct.unpack('h', file.read(struct.calcsize('h')))[0])
    print(struct.unpack('i', file.read(struct.calcsize('i')))[0])

1
2
3


### Persistence of objects (serialization) ... in disk

In [24]:
import pickle
list = ['red', 'green', 'blue']
pickle.dump(list, open('list.dat','wb'))

In [25]:
list2 = pickle.load(open('list.dat', 'rb'))
list2

['red', 'green', 'blue']

## [1] 4. Network

Using TCP:

In [26]:
# This code creates a TCP receiver daemon

import socket

PORT = 8001

class TCP_Receiver():

    # We use a context manager for implementing this "server"
    # (https://docs.python.org/3/reference/datamodel.html#context-managers).
    # This method is run when the "with" context is created (see below).
    def __enter__(self):
        '''Create a TCP socket.'''
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.bind(('', PORT))
        self.sock.listen(1)
        print('waiting for a connection at {} ...'\
              .format(self.sock.getsockname()))
        return self

    def accept(self):
        (self.peer_sock, peer) = self.sock.accept()
        print('\nconnection accepted from {}.'.format(peer))

    def receive(self):
        message_final_length = len('hello world!')
        message = bytearray()   # bytearray is mutable and therefore,
                                # faster appending than inmutable types
        while len(message) < message_final_length:
            chunk = self.peer_sock.recv(message_final_length - len(message))
            if chunk == b'':
                raise RuntimeError("socket connection broken")
            message.extend(chunk)
        #print('receivedddd "{}"'.format(message.decode("utf-8")))
        return message
    
    # This method is run when the "with" context is finised (see below).
    def __exit__(self,ext_type,exc_value,traceback):
        self.sock.close()
        print('socket closed')

def receiver_server():
    with TCP_Receiver() as recv:
        recv.accept()
        print('message = {}'.format(recv.receive().decode("utf-8")))
        
import threading

threading.Thread(target=receiver_server).start()

waiting for a connection at ('0.0.0.0', 8001) ...


In [27]:
# Now, we connect to the server.

# 1. Create a TCP socket 
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(('localhost', PORT))
print('connection established with {}'.format((sock.getpeername())))

# 2. Send a string to the server.
message = b'hello world!'
total_sent = 0 # bytes
while total_sent < len(message):
    sent = sock.send(message[total_sent:])
    if sent == 0:
        raise RuntimeError("socket connection broken")
    total_sent += sent
sock.close()

connection established with ('127.0.0.1', 8001)
connection accepted from ('127.0.0.1', 60446).

message = hello world!
socket closed


Using UDP:

In [28]:
import socket

PORT = 8001

class UDP_Receiver():

    # We use a context manager (https://docs.python.org/3/reference/datamodel.html#context-managers).
    def __enter__(self):
        '''Create a TCP socket.'''
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.sock.bind(('', PORT))
        print('listening at {} ...'\
              .format(self.sock.getsockname()))
        return self

    def receive(self):
        (message, from_addr) = self.sock.recvfrom(1024)
        print('received {} from {}'.format(message, from_addr))
        return message
    
    def __exit__(self,ext_type,exc_value,traceback):
        self.sock.close()
        print('socket closed')

def use_socket():
    with UDP_Receiver() as recv:
        print('message = {}'.format(recv.receive().decode("utf-8")))
        
import threading

threading.Thread(target=use_socket).start()

listening at ('0.0.0.0', 8001) ...


In [29]:
# Create a UDP socket 
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
message = b'hello world!'
sock.sendto(message, ('localhost', PORT))
sock.close()

received b'hello world!' from ('127.0.0.1', 60162)
message = hello world!
socket closed
